# Task A

## Example of Data Model (Reference)

### Climate  

`{
    {'_id': ObjectId('6287cc99c4c587048264cc0e'), 
    'station': 948702, 
    'date': datetime.datetime(2021, 12, 25, 0, 0), 
    'air_temperature_celcius': 17, 
    'relative_humidity': 50.4, 
    'windspeed_knots': 10.8, 
    'max_wind_speed': 16.9, 
    'precipitation ': ' 0.00I', 
    'GHI_w/m2': 145, 
    'precipitation_numeric': 0.0}
}`

### Hotspot  

`{
    '_id': ObjectId('628104241b875bcdc821215a'), 
    'latitude': -37.708, 
    'longitude': 145.1, 
    'datetime': '2021-12-25T04:29:08', 
    'confidence': 80, 
    'date': datetime.datetime(2021, 12, 25, 0, 0), 
    'surface_temperature_celcius': 54}
}`

## Justification 

The reason I used reference model is I think that for the climate data, we can re-use the same mongo database for the next 10 years since referencing model is scalable compares to embedding.Referencing model allow us to work with larger data compare to the embedding model since the embedding model require all data to be stored/wrapped within a dict, when referencing model allow us to work with different collection, so with referencing model we can work with larger dataset that store within 1 document. 

Other than that, referencing model is more suitable for one-to-many relationship according to the lecture. And the climate_historic dataset is the dataset where having all the date from 31st December 2020, which is the "one" in the one-to-many relationship, and the hotspot_historic dataset will be the "many" since there will be more than one row within the same day. 

Lastly, although referencing model require more complex query/work to be done when we want to have the attribute from both dataset, but I think for the climate, the number of operations to write to the data is significantly more than reading/query from the database. 

Hence, based on the 3 reason listed above I choose to use the referencing model.

In [1]:
import pymongo 
import pandas as pd
from datetime import datetime
from datetime import timezone
import dateutil.parser
from pymongo import MongoClient 
client = pymongo.MongoClient()
mydb = client.fit3182_assignment_db
climate_coll = mydb['climate']
hotspot_coll = mydb['hotspot']
climate_coll.drop()
hotspot_coll.drop()


In [2]:
climate_history = pd.read_csv('climate_historic.csv')
hotspot_history = pd.read_csv('hotspot_historic.csv')

In [3]:
climate_history.dtypes

station                      int64
date                        object
air_temperature_celcius      int64
relative_humidity          float64
windspeed_knots            float64
max_wind_speed             float64
precipitation               object
GHI_w/m2                     int64
dtype: object

In [4]:
hotspot_history.dtypes

latitude                       float64
longitude                      float64
datetime                        object
confidence                       int64
date                            object
surface_temperature_celcius      int64
dtype: object

In [5]:
# conversion of object to datetime 
climate_history['date'] = pd.to_datetime(climate_history['date'],dayfirst=True)

hotspot_history['date'] = pd.to_datetime(hotspot_history['date'],dayfirst=True)




In [6]:
# preprocess `precipitation` column 
climate_history['precipitation_numeric'] = climate_history['precipitation '].str.strip('IGAB')
climate_history['precipitation_numeric'] = climate_history['precipitation_numeric'].astype(str).astype(float)

In [7]:
climate_history.head()

,station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation,GHI_w/m2,precipitation_numeric
0,948700,2020-12-31,19,56.8,7.9,11.1,0.00I,154,0.00
1,948700,2021-01-02,15,50.7,9.2,13.0,0.02G,128,0.02
2,948700,2021-01-03,16,53.6,8.1,15.0,0.00G,133,0.00
3,948700,2021-01-04,24,61.6,7.7,14.0,0.00I,186,0.00
4,948700,2021-01-05,24,62.3,7.0,13.0,0.00I,185,0.00


In [9]:
climate_dict = climate_history.to_dict(orient='records')
hotspot_dict = hotspot_history.to_dict(orient='records')

In [10]:
print(climate_dict[0])
print(hotspot_dict[0])

{'station': 948700, 'date': Timestamp('2020-12-31 00:00:00'), 'air_temperature_celcius': 19, 'relative_humidity': 56.8, 'windspeed_knots': 7.9, 'max_wind_speed': 11.1, 'precipitation ': ' 0.00I', 'GHI_w/m2': 154, 'precipitation_numeric': 0.0}
{'latitude': -37.966, 'longitude': 145.05100000000002, 'datetime': '2021-12-27T04:16:51', 'confidence': 78, 'date': Timestamp('2021-12-27 00:00:00'), 'surface_temperature_celcius': 68}


In [11]:
climate_coll.insert_many(climate_dict)

In [79]:
for x in climate_coll.find():
    print(x)
    break

{'_id': ObjectId('628a4076c40eee0972357850'), 'station': 948700, 'date': datetime.datetime(2020, 12, 31, 0, 0), 'air_temperature_celcius': 19, 'relative_humidity': 56.8, 'windspeed_knots': 7.9, 'max_wind_speed': 11.1, 'precipitation ': ' 0.00I', 'GHI_w/m2': 154, 'precipitation_numeric': 0.0}


In [13]:
hotspot_coll.insert_many(hotspot_dict)

In [80]:
for x in hotspot_coll.find():
    print(x)
    break

{'_id': ObjectId('628a4077c40eee09723579be'), 'latitude': -37.966, 'longitude': 145.05100000000002, 'datetime': '2021-12-27T04:16:51', 'confidence': 78, 'date': datetime.datetime(2021, 12, 27, 0, 0), 'surface_temperature_celcius': 68}


# Part A Task 2

## a. Find climate data on 12th December 2021.

In [15]:
mq = {"date":datetime.strptime("12/12/2021","%d/%m/%Y")}
for x in climate_coll.find(mq): 
    print(x)

{'_id': ObjectId('628a4076c40eee09723579a9'), 'station': 948702, 'date': datetime.datetime(2021, 12, 12, 0, 0), 'air_temperature_celcius': 19, 'relative_humidity': 55.3, 'windspeed_knots': 6.2, 'max_wind_speed': 12.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 156, 'precipitation_numeric': 0.0}


## b. Find the latitude, longitude, surface temperature (°C), and confidence when the surface temperature (°C) was between 65 °C and 100 °C.

In [16]:
mq = {"surface_temperature_celcius":{"$gte": 65,"$lte":100}}
results = hotspot_coll.find(mq,{"latitude":1,"longitude":1,"confidence":1,"surface_temperature_celcius":1}).sort("surface_temperature_celcius")

for x in results:
    print(x)


{'_id': ObjectId('628a4077c40eee09723579f3'), 'latitude': -37.641999999999996, 'longitude': 149.263, 'confidence': 100, 'surface_temperature_celcius': 65}
{'_id': ObjectId('628a4077c40eee0972357a16'), 'latitude': -37.861999999999995, 'longitude': 144.175, 'confidence': 87, 'surface_temperature_celcius': 65}
{'_id': ObjectId('628a4077c40eee0972357a35'), 'latitude': -37.330999999999996, 'longitude': 143.122, 'confidence': 90, 'surface_temperature_celcius': 65}
{'_id': ObjectId('628a4077c40eee0972357a74'), 'latitude': -37.46, 'longitude': 148.102, 'confidence': 88, 'surface_temperature_celcius': 65}
{'_id': ObjectId('628a4077c40eee0972357a93'), 'latitude': -37.446, 'longitude': 148.102, 'confidence': 100, 'surface_temperature_celcius': 65}
{'_id': ObjectId('628a4077c40eee0972357ab3'), 'latitude': -37.396, 'longitude': 148.086, 'confidence': 100, 'surface_temperature_celcius': 65}
{'_id': ObjectId('628a4077c40eee0972357ae2'), 'latitude': -38.3998, 'longitude': 147.064, 'confidence': 89, 's

## c. Find date, surface temperature (°C), air temperature (°C), relative humidity and max wind speed on 15th and 16th of December 2021.

In [82]:
results = mydb.climate.aggregate([
    {
        "$lookup":{
            "from":"hotspot",
            "localField": "date",
            "foreignField": "date",
            "as":"hotspot"
        }
    },
    {
        "$match":{
            "$or":
            [
            {"date":datetime.strptime("15/12/2021","%d/%m/%Y")},
            {"date":datetime.strptime("16/12/2021","%d/%m/%Y")}
            ]
        }
    },
    {
        "$project":
        {
            "date":1, 
            "air_temperature_celcius":1, 
            "relative_humidity":1, 
            "max_wind_speed":1, 
            "hotspot":{"surface_temperature_celcius":1}
        }
    }
    
])
for x in results: 
    print (x)


{'_id': ObjectId('628a4076c40eee09723579ac'), 'date': datetime.datetime(2021, 12, 15, 0, 0), 'air_temperature_celcius': 18, 'relative_humidity': 52.0, 'max_wind_speed': 14.0, 'hotspot': [{'surface_temperature_celcius': 42}, {'surface_temperature_celcius': 36}, {'surface_temperature_celcius': 38}, {'surface_temperature_celcius': 40}]}
{'_id': ObjectId('628a4076c40eee09723579ad'), 'date': datetime.datetime(2021, 12, 16, 0, 0), 'air_temperature_celcius': 18, 'relative_humidity': 53.7, 'max_wind_speed': 13.0, 'hotspot': [{'surface_temperature_celcius': 43}, {'surface_temperature_celcius': 33}, {'surface_temperature_celcius': 54}, {'surface_temperature_celcius': 73}, {'surface_temperature_celcius': 55}, {'surface_temperature_celcius': 75}, {'surface_temperature_celcius': 55}, {'surface_temperature_celcius': 66}, {'surface_temperature_celcius': 56}, {'surface_temperature_celcius': 60}, {'surface_temperature_celcius': 73}, {'surface_temperature_celcius': 48}, {'surface_temperature_celcius': 5

## d. Find datetime, air temperature (°C), surface temperature (°C) and confidence when the confidence is between 80 and 100.

In [83]:
results = mydb.hotspot.aggregate([
    {
        "$lookup":{
            "from":"climate",
            "localField": "date",
            "foreignField": "date",
            "as":"climate"
        }
    },
    {
        "$match":
            {"confidence":{"$gte":80,"$lte":100}}
            
        
    },
    {
        "$project":
        {
            "datetime":1, 
            "surface_temperature_celcius":1, 
            "confidence":1, 
            "climate":{"air_temperature_celcius":1}
        }
    },
    {
        "$sort":{"confidence":1}
    }
    
])

for x in results: 
    print(x)

{'_id': ObjectId('628a4077c40eee09723579c2'), 'datetime': '2021-12-25T04:29:08', 'confidence': 80, 'surface_temperature_celcius': 54, 'climate': [{'air_temperature_celcius': 17}]}
{'_id': ObjectId('628a4077c40eee0972357a24'), 'datetime': '2021-11-11T13:30:08', 'confidence': 80, 'surface_temperature_celcius': 37, 'climate': [{'air_temperature_celcius': 18}]}
{'_id': ObjectId('628a4077c40eee0972357a47'), 'datetime': '2021-10-20T04:41:19', 'confidence': 80, 'surface_temperature_celcius': 46, 'climate': [{'air_temperature_celcius': 13}]}
{'_id': ObjectId('628a4077c40eee0972357a62'), 'datetime': '2021-10-04T04:41:02', 'confidence': 80, 'surface_temperature_celcius': 46, 'climate': [{'air_temperature_celcius': 19}]}
{'_id': ObjectId('628a4077c40eee0972357a63'), 'datetime': '2021-10-04T04:41:00', 'confidence': 80, 'surface_temperature_celcius': 46, 'climate': [{'air_temperature_celcius': 19}]}
{'_id': ObjectId('628a4077c40eee0972357abe'), 'datetime': '2021-09-23T04:59:11', 'confidence': 80, '

## e. Find the top 10 records with the highest surface temperature (°C).

In [84]:
results = hotspot_coll.aggregate([{"$sort":{"surface_temperature_celcius":-1}}, {"$limit":10}])

for x in results: 
    print(x)

{'_id': ObjectId('628a4077c40eee0972357e61'), 'latitude': -38.1665, 'longitude': 143.062, 'datetime': '2021-04-18T04:52:00', 'confidence': 100, 'date': datetime.datetime(2021, 4, 18, 0, 0), 'surface_temperature_celcius': 124}
{'_id': ObjectId('628a4077c40eee097235830b'), 'latitude': -36.343, 'longitude': 142.1986, 'datetime': '2021-04-04T04:32:50', 'confidence': 100, 'date': datetime.datetime(2021, 4, 4, 0, 0), 'surface_temperature_celcius': 123}
{'_id': ObjectId('628a4077c40eee0972357dd3'), 'latitude': -36.9318, 'longitude': 143.0907, 'datetime': '2021-05-01T04:14:20', 'confidence': 100, 'date': datetime.datetime(2021, 5, 1, 0, 0), 'surface_temperature_celcius': 122}
{'_id': ObjectId('628a4077c40eee09723583fa'), 'latitude': -37.016999999999996, 'longitude': 148.1297, 'datetime': '2021-03-18T03:50:50', 'confidence': 100, 'date': datetime.datetime(2021, 3, 18, 0, 0), 'surface_temperature_celcius': 121}
{'_id': ObjectId('628a4077c40eee0972357be5'), 'latitude': -34.9938, 'longitude': 141.

## f. Find the number of fires each day. You are required to only display the total number of fires and the date in the output.

In [21]:
results = mydb.climate.aggregate([
    {
        "$lookup":{
            "from":"hotspot",
            "localField": "date",
            "foreignField": "date",
            "as":"hotspot"
        }
    },
   
    {
        "$project":
        {
            "_id":0, 
            "date":1, 
            "num_of_fire":{'$size':'$hotspot'}
            
        }
    }
    
])
for x in results:
    print(x)

{'date': datetime.datetime(2020, 12, 31, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 2, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 3, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 4, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 5, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 6, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 7, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 8, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 9, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 10, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 11, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 12, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 13, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 14, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021, 1, 15, 0, 0), 'num_of_fire': 0}
{'date': datetime.datetime(2021,

## g. Find the records of fires where the confidence is below 70

In [85]:
mq = {"confidence":{"$lte": 70}}
results = hotspot_coll.find(mq,{"latitude":1,"longitude":1,"confidence":1,"surface_temperature_celcius":1}).sort("confidence")

for x in results:
    print(x)


{'_id': ObjectId('628a4077c40eee09723579db'), 'latitude': -37.986999999999995, 'longitude': 144.005, 'confidence': 50, 'surface_temperature_celcius': 38}
{'_id': ObjectId('628a4077c40eee0972357a0d'), 'latitude': -37.749, 'longitude': 148.297, 'confidence': 50, 'surface_temperature_celcius': 42}
{'_id': ObjectId('628a4077c40eee0972357a61'), 'latitude': -36.418, 'longitude': 141.596, 'confidence': 50, 'surface_temperature_celcius': 41}
{'_id': ObjectId('628a4077c40eee0972357a78'), 'latitude': -37.466, 'longitude': 148.1, 'confidence': 50, 'surface_temperature_celcius': 29}
{'_id': ObjectId('628a4077c40eee0972357aba'), 'latitude': -36.984, 'longitude': 148.25, 'confidence': 50, 'surface_temperature_celcius': 35}
{'_id': ObjectId('628a4077c40eee0972357af5'), 'latitude': -37.885999999999996, 'longitude': 147.207, 'confidence': 50, 'surface_temperature_celcius': 28}
{'_id': ObjectId('628a4077c40eee0972357af6'), 'latitude': -37.885999999999996, 'longitude': 147.207, 'confidence': 50, 'surface

## h. Find the average surface temperature (°C) for each day. You are required to only display average surface temperature (°C) and the date in the output.

In [86]:
results = mydb.climate.aggregate([
    {
        "$lookup":{
            "from":"hotspot",
            "localField": "date",
            "foreignField": "date",
            "as":"hotspot"
        }
    },
    {
        "$project":
        {
            "_id":0, 
            "date":1, 
            'AvgTemp': { '$avg': "$hotspot.surface_temperature_celcius"}}
            
        }    
])

for x in results: 
    print(x)

{'date': datetime.datetime(2020, 12, 31, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 2, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 3, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 4, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 5, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 6, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 7, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 8, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 9, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 10, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 11, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 12, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 13, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 14, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 15, 0, 0), 'AvgTemp': None}
{'date': datetime.datetime(2021, 1, 16, 0, 0), 

## i. Find the top 10 records with the lowest GHI.

In [87]:
results = climate_coll.aggregate([{"$sort":{"GHI_w/m2":-1}},{"$limit":10}])
for x in results:
    print(x)

{'_id': ObjectId('628a4076c40eee0972357856'), 'station': 948700, 'date': datetime.datetime(2021, 1, 7, 0, 0), 'air_temperature_celcius': 32, 'relative_humidity': 54.1, 'windspeed_knots': 12.8, 'max_wind_speed': 19.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 265, 'precipitation_numeric': 0.0}
{'_id': ObjectId('628a4076c40eee097235798d'), 'station': 948702, 'date': datetime.datetime(2021, 11, 14, 0, 0), 'air_temperature_celcius': 28, 'relative_humidity': 48.3, 'windspeed_knots': 11.9, 'max_wind_speed': 22.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 243, 'precipitation_numeric': 0.0}
{'_id': ObjectId('628a4076c40eee09723579aa'), 'station': 948702, 'date': datetime.datetime(2021, 12, 13, 0, 0), 'air_temperature_celcius': 28, 'relative_humidity': 49.9, 'windspeed_knots': 11.0, 'max_wind_speed': 19.0, 'precipitation ': ' 0.00I', 'GHI_w/m2': 240, 'precipitation_numeric': 0.0}
{'_id': ObjectId('628a4076c40eee0972357876'), 'station': 948700, 'date': datetime.datetime(2021, 2, 8, 0, 0), 'air_tempe

## j. Find the records with a 24-hour precipitation recorded between 0.20 to 0.35

In [88]:
mq = {"precipitation_numeric": {"$gte":0.2,"$lte":0.35 }}
results = climate_coll.find(mq,
                            {"station":1,"date":1,"air_temperature_celcius":1,"relative_humidity":1,"windspeed_knots":1,
                             "max_wind_speed":1,"precipitation ":1, "GHI_w/m2": 1})

for x in results: 
    print(x)

{'_id': ObjectId('628a4076c40eee097235785c'), 'station': 948700, 'date': datetime.datetime(2021, 1, 13, 0, 0), 'air_temperature_celcius': 19, 'relative_humidity': 54.1, 'windspeed_knots': 11.2, 'max_wind_speed': 18.1, 'precipitation ': ' 0.31G', 'GHI_w/m2': 157}
{'_id': ObjectId('628a4076c40eee09723578a7'), 'station': 948701, 'date': datetime.datetime(2021, 3, 29, 0, 0), 'air_temperature_celcius': 17, 'relative_humidity': 49.9, 'windspeed_knots': 12.2, 'max_wind_speed': 21.0, 'precipitation ': ' 0.24G', 'GHI_w/m2': 146}
{'_id': ObjectId('628a4076c40eee09723578bd'), 'station': 948701, 'date': datetime.datetime(2021, 4, 20, 0, 0), 'air_temperature_celcius': 20, 'relative_humidity': 53.5, 'windspeed_knots': 7.2, 'max_wind_speed': 15.9, 'precipitation ': ' 0.31G', 'GHI_w/m2': 166}
{'_id': ObjectId('628a4076c40eee09723578c3'), 'station': 948701, 'date': datetime.datetime(2021, 4, 26, 0, 0), 'air_temperature_celcius': 11, 'relative_humidity': 40.8, 'windspeed_knots': 12.2, 'max_wind_speed': 